In [1]:
# Copyright (c) Facebook, Inc. and its affiliates. All Rights Reserved
# -*- coding: utf-8 -*-
"""
Demo code

@author: Denis Tome'

"""
from torch.utils.data import DataLoader
import torch
from torchvision import transforms
from base import SetType

from mocap import Mocap
from utils import config, ConsoleLogger
from utils import evaluate, io
from model import TempModel
from tqdm import tqdm

LOGGER = ConsoleLogger("Main")
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

def main():
	"""Main"""

	LOGGER.info('Starting demo...')

	# ------------------- Data loader -------------------


	# let's load data from validation set as example
	data = Mocap(
		config.dataset.test,
		SetType.VAL,
		)
	data_loader = DataLoader(
		data,
		batch_size=config.data_loader.batch_size,
		shuffle=config.data_loader.shuffle)

	# ------------------- Evaluation -------------------

	eval_body = evaluate.EvalBody()
	eval_upper = evaluate.EvalUpperBody()
	eval_lower = evaluate.EvalLowerBody()

	# ------------------- Read dataset frames -------------------

	model = TempModel().to(device)
	model = torch.nn.DataParallel(model,device_ids=[0,1])
	model.eval()

	for it, test_batch in enumerate(tqdm(data_loader)):
		img = test_batch['image']
		p2d = test_batch['fisheye_joints_2d']
		p3d = test_batch['joints_3d_cam']
		action = test_batch['action']
		input_img = {'image':img}
		# LOGGER.info('Iteration: {}'.format(it))
		# LOGGER.info('Images: {}'.format(img.shape))
		# LOGGER.info('p2ds: {}'.format(p2d.shape))
		# LOGGER.info('p3ds: {}'.format(p3d.shape))
		# LOGGER.info('Actions: {}'.format(action))

		# -----------------------------------------------------------
		pred_output = model(input_img,0,False)
		# -----------------------------------------------------------

		# TODO: replace p3d_hat with model preditions
		p3d_hat = pred_output['regressor_dict']['pred_joint']

		# Evaluate results using different evaluation metrices
		y_output = p3d_hat.data.cpu().numpy() * 1000
		y_target = p3d.data.cpu().numpy() * 1000

		eval_body.eval(y_output, y_target, action)
		eval_upper.eval(y_output, y_target, action)
		eval_lower.eval(y_output, y_target, action)
		if (it+1)%200==0:
			print({'FullBody': eval_body.get_results(),
		   'UpperBody': eval_upper.get_results(),
		   'LowerBody': eval_lower.get_results()})
		# TODO: remove break
		

	# ------------------- Save results -------------------

	LOGGER.info('Saving evaluation results...')
	res = {'FullBody': eval_body.get_results(),
		   'UpperBody': eval_upper.get_results(),
		   'LowerBody': eval_lower.get_results()}

	io.write_json(config.eval.output_file, res)

	LOGGER.info('Done.')

In [3]:
main()

00:57:14 [INFO] Main: Starting demo...
  0%|          | 0/2676 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
  7%|▋         | 200/2676 [08:03<1:38:20,  2.38s/it]

{'FullBody': {'All': 69.7095718383789, 'Reacting': 61.00023651123047, 'UpperStretching': 70.50285339355469, 'Patting': 73.89226531982422, 'LowerStretching': 70.43629455566406, 'Walking': 72.92426300048828, 'Gaming': 64.06900787353516, 'Talking': 57.718685150146484, 'Gesticuling': 55.73611831665039, 'Greeting': 51.20087814331055}, 'UpperBody': {'All': 52.0637092590332, 'Reacting': 50.35930633544922, 'UpperStretching': 55.83163070678711, 'Patting': 55.94221496582031, 'LowerStretching': 43.67075729370117, 'Walking': 48.327945709228516, 'Gaming': 47.4422607421875, 'Talking': 47.18718719482422, 'Gesticuling': 48.24349594116211, 'Greeting': 43.30038833618164}, 'LowerBody': {'All': 87.35543060302734, 'Reacting': 71.64116668701172, 'UpperStretching': 85.1740951538086, 'Patting': 91.84231567382812, 'LowerStretching': 97.20182800292969, 'Walking': 97.52059173583984, 'Gaming': 80.69574737548828, 'Talking': 68.25018310546875, 'Gesticuling': 63.22872543334961, 'Greeting': 59.10136795043945}}


 15%|█▍        | 400/2676 [16:03<1:34:59,  2.50s/it]

{'FullBody': {'All': 69.51402282714844, 'Reacting': 60.90645217895508, 'UpperStretching': 70.01951599121094, 'Patting': 74.01456451416016, 'LowerStretching': 71.09854888916016, 'Walking': 72.78242492675781, 'Gaming': 66.48550415039062, 'Talking': 56.254676818847656, 'Gesticuling': 55.243011474609375, 'Greeting': 50.073429107666016}, 'UpperBody': {'All': 51.906097412109375, 'Reacting': 49.84092712402344, 'UpperStretching': 55.615787506103516, 'Patting': 56.62301254272461, 'LowerStretching': 43.89051055908203, 'Walking': 48.30354309082031, 'Gaming': 49.08641815185547, 'Talking': 46.34292221069336, 'Gesticuling': 46.47189712524414, 'Greeting': 41.255210876464844}, 'LowerBody': {'All': 87.12194061279297, 'Reacting': 71.97199249267578, 'UpperStretching': 84.42322540283203, 'Patting': 91.40609741210938, 'LowerStretching': 98.30657958984375, 'Walking': 97.26131439208984, 'Gaming': 83.88458251953125, 'Talking': 66.16642761230469, 'Gesticuling': 64.01412963867188, 'Greeting': 58.891639709472656

 22%|██▏       | 600/2676 [24:02<1:23:31,  2.41s/it]

...

 90%|████████▉ | 2400/2676 [1:36:03<11:21,  2.47s/it]

{'FullBody': {'All': 69.84127807617188, 'Reacting': 63.707035064697266, 'UpperStretching': 70.40111541748047, 'Patting': 70.31423950195312, 'LowerStretching': 71.89822387695312, 'Walking': 72.37203216552734, 'Gaming': 66.91883087158203, 'Talking': 54.670467376708984, 'Gesticuling': 55.594390869140625, 'Greeting': 47.9151611328125}, 'UpperBody': {'All': 52.126556396484375, 'Reacting': 52.27705383300781, 'UpperStretching': 55.808807373046875, 'Patting': 54.96131896972656, 'LowerStretching': 44.454654693603516, 'Walking': 47.92267990112305, 'Gaming': 49.208351135253906, 'Talking': 45.584774017333984, 'Gesticuling': 47.181365966796875, 'Greeting': 38.8099250793457}, 'LowerBody': {'All': 87.55599975585938, 'Reacting': 75.13701629638672, 'UpperStretching': 84.99341583251953, 'Patting': 85.66716003417969, 'LowerStretching': 99.341796875, 'Walking': 96.82140350341797, 'Gaming': 84.62931060791016, 'Talking': 63.756160736083984, 'Gesticuling': 64.00740814208984, 'Greeting': 57.0203971862793}}


 97%|█████████▋| 2600/2676 [1:44:04<03:08,  2.48s/it]

{'FullBody': {'All': 69.82882690429688, 'Reacting': 63.63347244262695, 'UpperStretching': 70.4097671508789, 'Patting': 70.33460235595703, 'LowerStretching': 71.87504577636719, 'Walking': 72.32540130615234, 'Gaming': 67.05892944335938, 'Talking': 54.62150573730469, 'Gesticuling': 55.49241638183594, 'Greeting': 47.91645812988281}, 'UpperBody': {'All': 52.13002395629883, 'Reacting': 52.19387435913086, 'UpperStretching': 55.82720947265625, 'Patting': 55.1173095703125, 'LowerStretching': 44.42859649658203, 'Walking': 47.885990142822266, 'Gaming': 49.257080078125, 'Talking': 45.587955474853516, 'Gesticuling': 47.190608978271484, 'Greeting': 38.88750076293945}, 'LowerBody': {'All': 87.52762603759766, 'Reacting': 75.07307434082031, 'UpperStretching': 84.99232482910156, 'Patting': 85.5519027709961, 'LowerStretching': 99.32149505615234, 'Walking': 96.76483154296875, 'Gaming': 84.86077117919922, 'Talking': 63.65505599975586, 'Gesticuling': 63.79421615600586, 'Greeting': 56.945411682128906}}


100%|██████████| 2676/2676 [1:47:06<00:00,  2.40s/it]
02:44:23 [INFO] Main: Saving evaluation results...
02:44:23 [INFO] Main: Done.
